<a href="https://colab.research.google.com/github/nhut-ngnn/OJT-Task/blob/main/Pytorch2ONNX/Task1_Convert2ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Jupyter Notebook is designed to train a simple CNN model using PyTorch and then convert it to the ONNX format for deployment. Below is a breakdown of its main components:

##**1. Installing Dependencies**

In [ ]:
!pip install onnx
!pip install onnxscript
!pip install onnxruntime
!pip install onnxruntime_tools

##**2. Importing Required Libraries**

In [ ]:
import time
import onnxruntime
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

##**3. Defining a CNN Model**

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


##**3. Preparing the MNIST Dataset**

* Downloads and loads the MNIST dataset.
* Applies normalization and transforms it to tensors.
* Creates data loaders for training and testing

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

##**4. Setting Up Training**

Defines:
* <code>device</code>: CPU for computation.
* <code>model</code>: Instance of MyModel.
* <code>criterion</code>: Cross-entropy loss function.
* <code>optimizer</code>: Adam optimizer.

In [ ]:
device = torch.device("cpu")
model = MyModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct/total:.2f}%")

model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f"Accuracy on test set: {100 * correct/total:.2f}%")

Epoch [1/5], Loss: 0.2591, Accuracy: 92.03%
Epoch [2/5], Loss: 0.0746, Accuracy: 97.63%
Epoch [3/5], Loss: 0.0550, Accuracy: 98.25%
Epoch [4/5], Loss: 0.0435, Accuracy: 98.66%
Epoch [5/5], Loss: 0.0363, Accuracy: 98.87%
Accuracy on test set: 98.60%


##**5. Training and ONNX Conversion**

Although not shown in the extracted code, it is expected that the notebook includes:
* Model training on the MNIST dataset.
* Conversion of the trained PyTorch model to ONNX format.
* Validation using <code>onnxruntime</code>.

In [ ]:

dummy_input = torch.randn(1, 1, 28, 28).to(device)
onnx_path = "MNIST_model.onnx"
torch.onnx.export(model, dummy_input, onnx_path,
                  input_names=["input"], output_names=["output"],
                  dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

print(f"Save completed at file path: {onnx_path}")


Save completed at file path: MNIST_model.onnx


In [ ]:
ort_session_original = onnxruntime.InferenceSession(onnx_path)

def inference_pytorch(model, images):
    model.eval()
    with torch.no_grad():
        return model(images.to(device)).cpu().numpy()

def inference_onnx(ort_session, images):
    images_np = images.cpu().numpy()
    ort_inputs = {"input": images_np}
    ort_outputs = ort_session.run(None, ort_inputs)
    return ort_outputs[0]

In [ ]:
import time
import numpy as np

def benchmark_model(inference_fn, model_name, num_runs=100):
    total_time = 0
    for _ in range(num_runs):
        images, _ = next(iter(test_loader))
        images = images.to(device)

        start_time = time.perf_counter()
        inference_fn(images)
        end_time = time.perf_counter()

        total_time += (end_time - start_time)

    avg_time = (total_time / num_runs) * 1000
    print(f"✅ {model_name} Average Inference Time: {avg_time:.4f} ms")

benchmark_model(lambda x: inference_pytorch(model, x), "PyTorch")
benchmark_model(lambda x: inference_onnx(ort_session_original, x), "ONNX")

✅ PyTorch Average Inference Time: 2.7227 ms
✅ ONNX Average Inference Time: 1.5038 ms


In [ ]:
import torch
import numpy as np
import onnxruntime

sample_image, sample_label = next(iter(test_loader))
sample_image = sample_image[:1].to(device)

torch_output = model(sample_image).detach().numpy()

onnx_output = ort_session_original.run(None, {"input": sample_image.numpy()})[0]

mse = np.mean((torch_output - onnx_output) ** 2)
print(f"MSE between PyTorch model and ONNX model: {mse}")

MSE between PyTorch model and ONNX model: 5.492495564266009e-13
Cosine similarity between PyTorch model and ONNX model: 1.0


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score

def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())

    return accuracy_score(all_labels, all_preds)

pytorch_acc = evaluate_model(model, test_loader)
print(f"PyTorch Accuracy: {pytorch_acc:.12f}")


PyTorch Accuracy: 0.986000000000


In [ ]:
def evaluate_onnx(ort_session, dataloader):
    all_preds, all_labels = [], []

    for images, labels in dataloader:
        ort_inputs = {"input": images.cpu().numpy()}
        ort_outputs = ort_session.run(None, ort_inputs)
        preds = np.argmax(ort_outputs[0], axis=1)
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

    return accuracy_score(all_labels, all_preds)

onnx_acc = evaluate_onnx(ort_session_original, test_loader)
print(f"ONNX Accuracy: {onnx_acc:.12f}")

ONNX Accuracy: 0.986000000000
